In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Getting started with Spark in BigQuery

## Overview

This notebook is a practical introduction to using [Google Cloud Serverless for Apache Spark](https://cloud.google.com/dataproc-serverless/docs/overview) within a Google Cloud environment. It demonstrates how to seamlessly launch an interactive Spark session from a [BigQuery Studio notebook](https://cloud.google.com/bigquery/docs/notebooks-introduction), connect to data stored in [BigQuery](https://cloud.google.com/bigquery/docs), and perform a straightforward data analysis workflow.

## Objectives

Using the [TheLook eCommerce](https://console.cloud.google.com/marketplace/product/bigquery-public-data/thelook-ecommerce) BigQuery public dataset, this notebook will guide you through finding the best-selling products.

By the end of this tutorial, you will know how to:
* Launch a [serverless Apache Spark](https://cloud.google.com/dataproc-serverless/docs/overview) session from a notebook using [Spark Connect](https://spark.apache.org/docs/latest/spark-connect-overview.html).
* Read data directly from a [BigQuery table](https://cloud.google.com/bigquery/docs/tables-intro) into a [Spark DataFrame](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/dataframe.html).
* Use [Spark SQL](https://spark.apache.org/docs/latest/sql-programming-guide.html) to join, aggregate, and analyze data.
* Convert a Spark DataFrame to a Pandas DataFrame to visualize your findings with [`matplotlib`](https://matplotlib.org/).

## Services and Costs

This tutorial uses the following billable components of Google Cloud:

* **BigQuery**: [Pricing](https://cloud.google.com/bigquery/pricing)

* **Colab Enterprise**: [Pricing](https://cloud.google.com/colab/pricing)

* **Dataproc**: [Pricing](https://cloud.google.com/dataproc/pricing)

You can use the [Pricing Calculator](https://cloud.google.com/products/calculator) to generate a cost estimate based on your projected usage.

# Create a Spark session

In [ ]:
from google.cloud.dataproc_spark_connect import DataprocSparkSession

# This will create a default Spark session
spark = DataprocSparkSession.builder.getOrCreate()

# If you would like to customize the Spark session, please refer to the
# documentation at https://cloud.google.com/bigquery/docs/use-spark
#
# For example, if you need to use a different subnetwork, use the code below

# from google.cloud.dataproc_v1 import Session

# session = Session()
# session.environment_config.execution_config.subnetwork_uri = "<subnetwork_name>"
# spark = DataprocSparkSession.builder.dataprocSessionConfig(session).getOrCreate()

/usr/local/lib/python3.12/dist-packages/google/cloud/dataproc_spark_connect/session.py:332: UserWarning: Python version mismatch detected: Client is using Python 3.12, but Dataproc runtime 3.0 uses Python 3.11. This mismatch may cause issues with Python UDF (User Defined Function) compatibility. Consider using Python 3.11 for optimal UDF execution.
  dataproc_config: Session = self._get_dataproc_config()


██████████████████████████████████████████▏                                     

In [ ]:
import pandas as pd
import requests
import io
from datetime import datetime, timedelta

In [ ]:
def buscar_serie_temporal_bcb(codigo_serie, nome_coluna, data_inicio="01/01/2010"):
    """
    Busca uma série temporal no Banco Central do Brasil (BCB) via API do SGS.

    Args:
        codigo_serie (int): Código da série no SGS do BCB.
        nome_coluna (str): Nome a ser dado à coluna de dados no DataFrame.
        data_inicio (str): Data de início da busca no formato 'dd/mm/aaaa'.

    Returns:
        pd.DataFrame: DataFrame com as colunas 'ano_mes' e a série de dados.
    """
    # URL da API do SGS do BCB
    url = f"https://api.bcb.gov.br/dados/serie/bcdata.sgs.{codigo_serie}/dados?formato=json&dataInicial={data_inicio}"

    try:
        response = requests.get(url)
        response.raise_for_status()
        dados = response.json()

    except requests.exceptions.RequestException as e:
        print(f"Erro ao buscar a série {nome_coluna} (Código {codigo_serie}): {e}")
        return pd.DataFrame()

    if not dados:
        print(f"A série {nome_coluna} (Código {codigo_serie}) retornou dados vazios.")
        return pd.DataFrame()

    # Cria o DataFrame a partir do JSON
    df = pd.DataFrame(dados)

    # Renomeia as colunas
    df.rename(columns={'valor': nome_coluna, 'data': 'data_completa'}, inplace=True)

    # Converte 'data' para o formato datetime
    df['data_completa'] = pd.to_datetime(df['data_completa'], format='%d/%m/%Y')

    # Cria a coluna 'ano_mes' no formato YYYY-MM
    df['ano_mes'] = df['data_completa'].dt.strftime('%Y-%m')

    # Converte valores para numérico
    df[nome_coluna] = pd.to_numeric(df[nome_coluna], errors='coerce')

    # ✅ SOLUÇÃO: Agrupa por ano_mes e pega a MÉDIA (ou último valor)
    # Para séries diárias, isso calcula a média mensal
    # Para séries já mensais, mantém o valor único
    df_mensal = df.groupby('ano_mes', as_index=False).agg({
        nome_coluna: 'mean'  # Usa 'mean' para média ou 'last' para último valor do mês
    })

    return df_mensal

In [ ]:
# --- Códigos das Séries do SGS do BCB (Mensais) ---
# Você pode buscar outros códigos na página do BCB/SGS
SERIES_BCB = {
    # === INDICADORES DE CUSTO ===
    4390: 'selic_meta_mensal',  # Taxa SELIC (custo de capital)

    # === INFLAÇÃO (afeta custos e margens) ===
    433: 'ipca_acumulado_12m',  # IPCA acumulado 12 meses
    13522: 'ipca_mensal',  # IPCA mensal (variação mais imediata)

    # === CRÉDITO E INADIMPLÊNCIA ===
    21082: 'inadimplencia_pj_livre',  # Inadimplência PJ - Recursos Livres
    20542: 'volume_credito_pj_total',  # Volume de crédito PJ total (R$ milhões)
    20714: 'spread_credito_pj',  # Spread médio das operações de crédito PJ

    # === CÂMBIO ===
    10813: 'cambio_dolar_media_mensal',  # Dólar - Média mensal de venda

    # === ATIVIDADE ECONÔMICA ===
    24363: 'ibc_br_dessazonalizado',  # IBC-Br (proxy do PIB mensal)

    # === CONFIANÇA E EXPECTATIVAS ===
    4394: 'icei',  # Índice de Confiança Empresarial (FGV)
    7341: 'nivel_utilizacao_capacidade',  # Nível de Utilização da Capacidade Instalada - Indústria

    # === MERCADO DE TRABALHO ===
    24369: 'taxa_desemprego',  # Taxa de desemprego (PNAD Contínua)

    # === MASSA SALARIAL (poder de compra) ===
    28089: 'massa_salarial_real',  # Massa de rendimento real habitual
}

In [ ]:
def coletar_indicadores_economicos():

    DATA_INICIO_COLETA = "01/01/2016"

    dfs_indicadores = []

    for codigo, nome in SERIES_BCB.items():
        print(f"Coletando série: {nome} (Código: {codigo})...")
        df_serie = buscar_serie_temporal_bcb(codigo, nome, DATA_INICIO_COLETA)
        if not df_serie.empty:
            dfs_indicadores.append(df_serie)

    if not dfs_indicadores:
        print("Nenhuma série foi coletada com sucesso.")
        return pd.DataFrame()

    # 1. Combina todos os DataFrames em um único
    df_final = dfs_indicadores[0]
    for i in range(1, len(dfs_indicadores)):
        df_final = pd.merge(df_final, dfs_indicadores[i], on='ano_mes', how='outer')

    # 2. Converte todas as colunas de valor para tipo numérico
    for col in df_final.columns:
        if col != 'ano_mes':
            df_final[col] = pd.to_numeric(df_final[col], errors='coerce')

    # 3. Ordena o DataFrame por ano_mes
    df_final.sort_values(by='ano_mes', inplace=True)
    df_final.reset_index(drop=True, inplace=True)

    # ✅ VERIFICAÇÃO ADICIONAL: Remove duplicatas caso ainda existam
    df_final = df_final.drop_duplicates(subset=['ano_mes'], keep='first')

    print("\nColeta de Indicadores Econômicos Finalizada.")
    print(f"DataFrame Final (Shape: {df_final.shape}):")
    print(f"Período: {df_final['ano_mes'].min()} a {df_final['ano_mes'].max()}")
    print(f"Total de meses únicos: {df_final['ano_mes'].nunique()}")

    return df_final

In [ ]:

# Chama a função para obter o DataFrame final
df_indicadores = coletar_indicadores_economicos()

# Exibe o resultado
df_indicadores.tail()

Coletando série: selic_meta_mensal (Código: 4390)...
Coletando série: ipca_acumulado_12m (Código: 433)...
Coletando série: ipca_mensal (Código: 13522)...
Coletando série: inadimplencia_pj_livre (Código: 21082)...
Coletando série: volume_credito_pj_total (Código: 20542)...
Coletando série: spread_credito_pj (Código: 20714)...
Coletando série: cambio_dolar_media_mensal (Código: 10813)...
Coletando série: ibc_br_dessazonalizado (Código: 24363)...
Coletando série: icei (Código: 4394)...
Coletando série: nivel_utilizacao_capacidade (Código: 7341)...
Coletando série: taxa_desemprego (Código: 24369)...
Coletando série: massa_salarial_real (Código: 28089)...
Erro ao buscar a série massa_salarial_real (Código 28089): Expecting value: line 1 column 1 (char 0)

Coleta de Indicadores Econômicos Finalizada.
DataFrame Final (Shape: (119, 12)):
Período: 2016-01 a 2025-11
Total de meses únicos: 119


,ano_mes,selic_meta_mensal,ipca_acumulado_12m,ipca_mensal,inadimplencia_pj_livre,volume_credito_pj_total,spread_credito_pj,cambio_dolar_media_mensal,ibc_br_dessazonalizado,icei,nivel_utilizacao_capacidade,taxa_desemprego
114,2025-07,1.28,0.26,5.23,3.78,3862382.0,31.58,5.527887,113.06029,102.7,47.3,5.6
115,2025-08,1.16,-0.11,5.13,3.94,3895526.0,31.70,5.446324,110.37876,108.1,NaN,5.6
116,2025-09,1.22,0.48,5.17,3.90,3938846.0,31.36,5.366809,109.24425,105.8,NaN,5.6
117,2025-10,1.28,0.09,4.68,NaN,NaN,NaN,5.384926,NaN,NaN,47.2,NaN
118,2025-11,0.72,NaN,NaN,NaN,NaN,NaN,5.326700,NaN,NaN,NaN,NaN


In [ ]:
df_spark = spark.createDataFrame(df_indicadores)

In [ ]:
print("Spark DataFrame Schema:")
df_spark.printSchema()
df_spark.show(5)

Spark DataFrame Schema:
root
 |-- ano_mes: string (nullable = true)
 |-- selic_meta_mensal: double (nullable = true)
 |-- ipca_acumulado_12m: double (nullable = true)
 |-- ipca_mensal: double (nullable = true)
 |-- inadimplencia_pj_livre: double (nullable = true)
 |-- volume_credito_pj_total: double (nullable = true)
 |-- spread_credito_pj: double (nullable = true)
 |-- cambio_dolar_media_mensal: double (nullable = true)
 |-- ibc_br_dessazonalizado: double (nullable = true)
 |-- icei: double (nullable = true)
 |-- nivel_utilizacao_capacidade: double (nullable = true)
 |-- taxa_desemprego: double (nullable = true)



|           0/0 Tasks

+-------+-----------------+------------------+-----------+----------------------+-----------------------+-----------------+-------------------------+----------------------+-----+---------------------------+---------------+
|ano_mes|selic_meta_mensal|ipca_acumulado_12m|ipca_mensal|inadimplencia_pj_livre|volume_credito_pj_total|spread_credito_pj|cambio_dolar_media_mensal|ibc_br_dessazonalizado| icei|nivel_utilizacao_capacidade|taxa_desemprego|
+-------+-----------------+------------------+-----------+----------------------+-----------------------+-----------------+-------------------------+----------------------+-----+---------------------------+---------------+
|2016-01|             1.06|              1.27|      10.71|                  3.47|              1618298.0|            31.03|                 4.051715|               90.3646|57.08|                       36.6|            9.6|
|2016-02|              1.0|               0.9|      10.36|                   3.5|              1606589.0|   

In [ ]:
# Substitua pelos seus valores
PROJECT_ID = "trabalho-final-pdm-478021"
DATASET_ID = "main_database"
TABLE_ID = "indicadores_economicos_bronze"
FULL_TABLE_REF = f"{DATASET_ID}.{TABLE_ID}"

# Bucket do GCS usado como intermediário temporário (Obrigatório para o Conector)
# No BigQuery Studio, geralmente um bucket temporário já é configurado.
# Se precisar forçar, use apenas o nome do bucket:
GCS_TEMP_BUCKET = "bq-spark-staging-bucket" # Ex: bq-spark-staging-bucket-12345

# Processo de Escrita
try:
    (
        df_spark.write
        .format("bigquery") # Especifica o formato BigQuery
        #.option("temporaryGcsBucket", GCS_TEMP_BUCKET) # OBRIGATÓRIO: Bucket de staging
        .option("parentProject", PROJECT_ID) # Opcional, mas boa prática
        .option("table", FULL_TABLE_REF) # O nome completo da tabela (dataset.table)

        # Modo de escrita:
        # "overwrite": Sobrescreve a tabela (cuidado!)
        # "append": Adiciona novas linhas
        # "errorifexists": (Padrão) Gera erro se a tabela já existe
        .mode("overwrite")

        .save()
    )
    print(f"\nSucesso: Dados salvos na tabela BigQuery: {FULL_TABLE_REF}")

except Exception as e:
    print(f"\nErro ao salvar no BigQuery: {e}")
    print("Verifique se o bucket GCS e a tabela de destino têm as permissões corretas.")

  0%|           0/8 Tasks


Sucesso: Dados salvos na tabela BigQuery: main_database.indicadores_economicos_bronze


## Adicionando tabela Silver

In [ ]:
df_indicadores_silver = df_indicadores.fillna(df_indicadores.mean(numeric_only=True))
df_indicadores_silver.tail()

,ano_mes,selic_meta_mensal,ipca_acumulado_12m,ipca_mensal,inadimplencia_pj_livre,volume_credito_pj_total,spread_credito_pj,cambio_dolar_media_mensal,ibc_br_dessazonalizado,icei,nivel_utilizacao_capacidade,taxa_desemprego
114,2025-07,1.28,0.260000,5.230000,3.780000,3.862382e+06,31.580000,5.527887,113.060290,102.700000,47.3000,5.60000
115,2025-08,1.16,-0.110000,5.130000,3.940000,3.895526e+06,31.700000,5.446324,110.378760,108.100000,52.9075,5.60000
116,2025-09,1.22,0.480000,5.170000,3.900000,3.938846e+06,31.360000,5.366809,109.244250,105.800000,52.9075,5.60000
117,2025-10,1.28,0.090000,4.680000,3.128291,2.455905e+06,26.768803,5.384926,98.944508,85.576068,47.2000,10.74188
118,2025-11,0.72,0.420508,5.464322,3.128291,2.455905e+06,26.768803,5.326700,98.944508,85.576068,52.9075,10.74188


In [ ]:
df_spark_silver = spark.createDataFrame(df_indicadores_silver)

In [ ]:
print("Spark DataFrame Schema:")
df_spark_silver.printSchema()
df_spark_silver.show(5)

Spark DataFrame Schema:
root
 |-- ano_mes: string (nullable = true)
 |-- selic_meta_mensal: double (nullable = true)
 |-- ipca_acumulado_12m: double (nullable = true)
 |-- ipca_mensal: double (nullable = true)
 |-- inadimplencia_pj_livre: double (nullable = true)
 |-- volume_credito_pj_total: double (nullable = true)
 |-- spread_credito_pj: double (nullable = true)
 |-- cambio_dolar_media_mensal: double (nullable = true)
 |-- ibc_br_dessazonalizado: double (nullable = true)
 |-- icei: double (nullable = true)
 |-- nivel_utilizacao_capacidade: double (nullable = true)
 |-- taxa_desemprego: double (nullable = true)



|           0/0 Tasks

+-------+-----------------+------------------+-----------+----------------------+-----------------------+-----------------+-------------------------+----------------------+-----+---------------------------+---------------+
|ano_mes|selic_meta_mensal|ipca_acumulado_12m|ipca_mensal|inadimplencia_pj_livre|volume_credito_pj_total|spread_credito_pj|cambio_dolar_media_mensal|ibc_br_dessazonalizado| icei|nivel_utilizacao_capacidade|taxa_desemprego|
+-------+-----------------+------------------+-----------+----------------------+-----------------------+-----------------+-------------------------+----------------------+-----+---------------------------+---------------+
|2016-01|             1.06|              1.27|      10.71|                  3.47|              1618298.0|            31.03|                 4.051715|               90.3646|57.08|                       36.6|            9.6|
|2016-02|              1.0|               0.9|      10.36|                   3.5|              1606589.0|   

In [ ]:
# Substitua pelos seus valores
PROJECT_ID = "trabalho-final-pdm-478021"
DATASET_ID = "main_database"
TABLE_ID = "indicadores_economicos_silver"
FULL_TABLE_REF = f"{DATASET_ID}.{TABLE_ID}"

# Bucket do GCS usado como intermediário temporário (Obrigatório para o Conector)
# No BigQuery Studio, geralmente um bucket temporário já é configurado.
# Se precisar forçar, use apenas o nome do bucket:
GCS_TEMP_BUCKET = "bq-spark-staging-bucket" # Ex: bq-spark-staging-bucket-12345

# Processo de Escrita
try:
    (
        df_spark_silver.write
        .format("bigquery") # Especifica o formato BigQuery
        #.option("temporaryGcsBucket", GCS_TEMP_BUCKET) # OBRIGATÓRIO: Bucket de staging
        .option("parentProject", PROJECT_ID) # Opcional, mas boa prática
        .option("table", FULL_TABLE_REF) # O nome completo da tabela (dataset.table)

        # Modo de escrita:
        # "overwrite": Sobrescreve a tabela (cuidado!)
        # "append": Adiciona novas linhas
        # "errorifexists": (Padrão) Gera erro se a tabela já existe
        .mode("overwrite")

        .save()
    )
    print(f"\nSucesso: Dados salvos na tabela BigQuery: {FULL_TABLE_REF}")

except Exception as e:
    print(f"\nErro ao salvar no BigQuery: {e}")
    print("Verifique se o bucket GCS e a tabela de destino têm as permissões corretas.")

  0%|           0/8 Tasks


Sucesso: Dados salvos na tabela BigQuery: main_database.indicadores_economicos_silver


In [ ]:
# Stop the Spark session and release all resources
spark.stop()

# Summary & next steps

This notebook provided a hands-on tour of how to use Spark in a BigQuery Studio notebook to connect to BigQuery, perform data analysis, and visualize the results.

## What we've covered:

* **Spark session creation**: We initialized a serverless Spark session allowing us to run Spark code interactively.
* **Loading BigQuery data**: We loaded tables directly from the BigQuery public dataset `thelook_ecommerce` into Spark DataFrames.
* **Spark SQL analysis**: We used `spark.sql()` to join the `products` and `order_items` tables, aggregate the data, and identify the top 20 best-selling products.
* **Visualization**: We converted the final Spark DataFrame into a Pandas DataFrame to easily plot the results using `matplotlib`.
* **Resource cleanup**: We concluded by stopping the Spark session to release all the underlying cloud resources and prevent further costs.

## Next steps

To continue building your skills, here are some excellent resources:

* **Explore Spark on Google Cloud**: Dive deeper into the [documentation for using Spark on Google Cloud](https://cloud.google.com/solutions/spark) to learn about more advanced configurations and features.
* **PySpark Documentation**: Familiarize yourself with the full range of capabilities in the [official PySpark documentation](https://spark.apache.org/docs/latest/api/python/).
* **Analyze Another Dataset**: Apply what you've learned to a different dataset from the [BigQuery Public Datasets](https://cloud.google.com/bigquery/public-data) collection.
* **Spark Connect**: Learn more about the [client-server architecture](https://spark.apache.org/docs/latest/spark-connect-overview.html) powering BigQuery's integration with Spark.